<a href="https://colab.research.google.com/github/aminghafari1/lcaging-fmri/blob/main/Copy_of_stimulus_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
## This code is for reading the subject data and trying to get the EV files responsible for the visual and auditory stimuli.
## The outline is that the subject number is entered and the resulting output is EV files for all the contrasts tha are considered.
## For now I'm gonna only focus on separating the effect of the grip from the V/A stimuli, then we will see what to do with the 
## different stimulus/handgrip levels and also the subject answers and the confidence ratings.

from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
import glob
import os
def substring_after(s,delim):
  return s.partition(delim)[2]
def substring_before(s,delim):
  return s.partition(delim)[0]
import scipy.io as sio
def remove_duplicates_and_sort(vector):
    unique_vector = list(set(vector))
    sorted_vector = sorted(unique_vector)
    return sorted_vector


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [22]:
sub_num=input("Please enter the subject number:")
print("Working on the visual and auditory data of sub %s now." %(sub_num))

base_dir=glob.glob('/content/gdrive/Shareddrives/LC-Aging/'\
  'Older Adult - MRI Study/Data Collection/BAP/BAP data/')
base_dir=base_dir[0]
sub_folder_name="sub-BAP"+sub_num
z=os.listdir(base_dir)
print("The folder we are looking for here is: %s" %(sub_folder_name))
while sub_num != "quit":
    if sub_folder_name in z:
        print("The folder %s was found in the list." %(sub_folder_name) )
        break
    else:
        print("The folder %s was not found in the list." %(sub_folder_name) )
        print("Please enter another subject code or type 'quit' to exit.")
    sub_num = input("Please enter a subject number or type 'quit' to exit: ")
subject_folder=base_dir+sub_folder_name
print(subject_folder)
sessions=[2,3]
for session_number in sessions:
  temp="/ses-%s/InsideScanner" %(session_number)
  scans_folder=subject_folder+temp
  scans_contents=os.listdir(scans_folder)
  mat_files= [element for element in scans_contents if element.endswith('.mat') and "eyetrack" not in element and "MVCnPRAC" not in element]
  temp_perf=mat_files[0]
  if "Voddball" in temp_perf:
    visual_files=mat_files
    visual_scans_folder=scans_folder+"/"
  else:
    audio_files=mat_files
    audio_scans_folder=scans_folder+"/"
  
## Finding the save directory
res_directory="BAP"+str(sub_num)
print(res_directory)
save_dir_vis='/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s/visual/ev_files' %(res_directory)
save_dir_aud='/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s/auditory/ev_files' %(res_directory)
if not os.path.exists(save_dir_vis):
  os.makedirs(save_dir_vis)
if not os.path.exists(save_dir_aud):
  os.makedirs(save_dir_aud)

Please enter the subject number:108
Working on the visual and auditory data of sub 108 now.
The folder we are looking for here is: sub-BAP108
The folder sub-BAP108 was found in the list.
/content/gdrive/Shareddrives/LC-Aging/Older Adult - MRI Study/Data Collection/BAP/BAP data/sub-BAP108
BAP108


In [33]:

## Because in some of the subjects the number of auditory tasks might differ from the visual ones
## we need to exploit 2 for loops, one for each here.
## This section is only for the visual test
number_of_trials_per_run=30
visual_ones=np.ones(number_of_trials_per_run)
grip_ones=np.ones(number_of_trials_per_run)
for current_file in visual_files:
  task_file=current_file
  file_directory=visual_scans_folder+task_file
  temp=substring_after(file_directory,"run")
  run_index=temp[0]
  print("Now reading run number %s of the visual test." %(run_index))
  current_run=sio.loadmat(file_directory)
  experiment_start_time=current_run['ExpStartTimeP']
  trial_start=current_run['stimulusStartTimeP']-experiment_start_time ##This is also grip start time
  grip_end_time=current_run['relaxStartTimeP']-experiment_start_time
  first_stimulus_start=current_run['SoundStartTimeP']-experiment_start_time
  first_stimulus_end=current_run['g1_OffsetTimeP']-experiment_start_time
  second_stimulus_end=current_run['g2_OffsetTimeP']-experiment_start_time
  second_stimulus_start=current_run['g2_OnsetTimeP']-experiment_start_time
  grip_force=current_run['isStrengthHi']
  button_response=current_run['ButtonResponse']
  stimlev=current_run['StimLev']
  mixtr=current_run['mixtr']
  slice_order=mixtr[:,0]
  slice_index=slice_order-1
  visual_presented_order=visual_ev[slice_index]
  stimlev=stimlev.squeeze()
  button_response=button_response.squeeze() ##same for 3, different for 1
  grip_force=grip_force.squeeze()
  grip_length=grip_end_time-trial_start
  first_stimulus_length=first_stimulus_end-first_stimulus_start
  second_stimulus_length=second_stimulus_end-second_stimulus_start
  visual_first_vector=np.zeros(np.shape(first_stimulus_start)[1])
  visual_second_vector=np.zeros(np.shape(first_stimulus_start)[1])
  visual_third_vector=np.zeros(np.shape(first_stimulus_start)[1])
  grip_first_vector=np.zeros(np.shape(trial_start)[1])
  grip_second_vector=np.zeros(np.shape(trial_start)[1])
  j=0
  num_cols=3
  same_matrix=np.empty((0, num_cols))
  diff_matrix=np.empty((0, num_cols))
  lowgrip_matrix=np.empty((0,num_cols))
  highgrip_matrix=np.empty((0,num_cols))

  for i in range(np.shape(first_stimulus_start)[1]):
    grip_first_vector[i]=trial_start[0,i]
    grip_second_vector[i]=grip_length[0,i]
    visual_first_vector[i]=first_stimulus_start[0,i]
    visual_second_vector[i]=second_stimulus_end[0,i]-first_stimulus_start[0,i]
    visual_third_vector[i]=stimlev[i]
    if grip_force[i]==0:
      row=[grip_first_vector[i],grip_second_vector[i],1]
      lowgrip_matrix=np.vstack([lowgrip_matrix,row])
    if grip_force[i]==1:
      row=[grip_first_vector[i],grip_second_vector[i],1]
      highgrip_matrix=np.vstack([highgrip_matrix,row])
    if button_response[i]==3:
      row=[visual_first_vector[i],visual_second_vector[i],1]
      same_matrix=np.vstack([same_matrix,row])
    if button_response[i]==1:
      row=[visual_first_vector[i],visual_second_vector[i],1]
      diff_matrix=np.vstack([diff_matrix,row])
  visual_ev=np.column_stack((visual_first_vector,visual_second_vector,visual_third_vector)) 
  grip_ev=np.column_stack((grip_first_vector,grip_second_vector,grip_ones)) 
  run_save_dir=save_dir_vis+"/run"+run_index
  fmt = ['%.4f', '%.4f', '%.d']
  dmt = ['%.4f', '%.4f', '%.2f']
  if not os.path.exists(run_save_dir):
    os.makedirs(run_save_dir)
  ## oddball contrasts
  stim_sorted=remove_duplicates_and_sort(stimlev)
  oddballs=np.zeros((5,6,3))
  for j in range(len(stim_sorted)):
    filename=f"oddball_lev_{j+1}.txt"
    k=0
    for row in visual_presented_order:
      if row[2]==stim_sorted[j]:
        oddballs[j][k][:]=row
        k=k+1
    oddball_ev=os.path.join(run_save_dir,filename)
    np.savetxt(oddball_ev,oddballs[j], delimiter=" ", fmt=dmt)

  oddball_order=np.zeros((len(slice_order)))


  ##oddball contrasts
  
  visual_filename=f"visual_run_{run_index}.txt"
  grip_filename=f"visual_grip_run_{run_index}.txt"
  same_filename=f"stimuli_same_run_{run_index}.txt"
  diff_filename=f"stimuli_diff_run_{run_index}.txt"
  lowgrip_filename=f"low_grip_run_{run_index}.txt"
  highgrip_filename=f"high_grip_run_{run_index}.txt"
  ev1_address=os.path.join(run_save_dir,visual_filename)
  ev2_address=os.path.join(run_save_dir,grip_filename)
  ev3_address=os.path.join(run_save_dir,same_filename)
  ev4_address=os.path.join(run_save_dir,diff_filename)
  ev5_address=os.path.join(run_save_dir,lowgrip_filename)
  ev6_address=os.path.join(run_save_dir,highgrip_filename)
  np.savetxt(ev1_address,visual_ev, delimiter=" ",fmt=dmt)
  np.savetxt(ev2_address,grip_ev, delimiter=" ",fmt=fmt)
  np.savetxt(ev3_address,same_matrix,delimiter=" ",fmt=fmt)
  np.savetxt(ev4_address,diff_matrix,delimiter=" ",fmt=fmt)
  np.savetxt(ev5_address,lowgrip_matrix,delimiter=" ",fmt=fmt)
  np.savetxt(ev6_address,highgrip_matrix,delimiter=" ",fmt=fmt)



Now reading run number 1 of the visual test.
Now reading run number 2 of the visual test.
Now reading run number 3 of the visual test.
Now reading run number 4 of the visual test.
Now reading run number 5 of the visual test.


In [42]:
slice_order
juv=slice_order-1

visual_juv=visual_ev[juv]

In [43]:
visual_juv


array([[4.22617661e+01, 7.00062800e-01, 0.00000000e+00],
       [5.45121774e+01, 7.00424100e-01, 0.00000000e+00],
       [2.45261079e+02, 7.00069300e-01, 6.00000000e-02],
       [3.04763653e+02, 7.00238300e-01, 6.00000000e-02],
       [2.01509126e+02, 7.00026000e-01, 1.20000000e-01],
       [2.17260073e+02, 6.99567800e-01, 2.40000000e-01],
       [3.73006884e+02, 7.00312000e-01, 1.20000000e-01],
       [3.46755656e+02, 6.99921800e-01, 4.80000000e-01],
       [6.85131728e+01, 6.99837500e-01, 0.00000000e+00],
       [1.35006213e+02, 6.99647300e-01, 6.00000000e-02],
       [9.30141002e+01, 6.99748300e-01, 1.20000000e-01],
       [1.47256769e+02, 6.99957900e-01, 1.20000000e-01],
       [3.59006284e+02, 6.99850500e-01, 6.00000000e-02],
       [3.79997150e+00, 7.00088700e-01, 0.00000000e+00],
       [1.89258203e+02, 7.00439400e-01, 6.00000000e-02],
       [3.88757539e+02, 7.00073700e-01, 2.40000000e-01],
       [2.31260345e+02, 7.00221200e-01, 4.80000000e-01],
       [3.32765316e+02, 6.99620

In [ ]:
## oddball discriminator, run this cell if you want to get different ev files based on the oddball level
oddballs=np.zeros((5,6,3))
stim_sorted=remove_duplicates_and_sort(stimlev)
for j in range(len(stim_sorted)):
  k=0
  for row in visual_ev:
    if row[2]==stim_sorted[j]:
      oddballs[j][k][:]=row
      k=k+1
      ## You need 2 for looops.
for i in range(np.shape(oddballs)[0]):
  run_index=i+1
  filename=f"oddball_level_run{run_index}.txt"
  run_save_dir=save_dir_vis+"/run"+str(run_index)
  ev_address=os.path.join(run_save_dir,filename)
  print("The current ev address is: %s" %(ev_address))
  np.savetxt(ev_address,oddballs[i],delimiter= "" , fmt=fmt)


The current ev address is: /content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/BAP103/visual/ev_files/run1/oddball_level_run1.txt
The current ev address is: /content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/BAP103/visual/ev_files/run2/oddball_level_run2.txt
The current ev address is: /content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/BAP103/visual/ev_files/run3/oddball_level_run3.txt
The current ev address is: /content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/BAP103/visual/ev_files/run4/oddball_level_run4.txt
The current ev address is: /content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/BAP103/visual/ev_files/run5/oddball_level_run5.txt


In [ ]:
ev_address

'/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/BAP103/visual/ev_files/run5/oddball_level_run5.txt'

In [ ]:
## Use this in case that you want to delete contents of one directory
now=substring_before(save_dir_vis,"/ev")
print(save_dir_vis)
now
txt_files = glob.glob(os.path.join(save_dir_vis, '*.txt'))
for file in txt_files:
    os.remove(file)
    print(f"Deleted file: {file}")


array([[[7.19892549e+01, 6.96532400e-01, 0.00000000e+00],
        [7.19892549e+01, 6.96532400e-01, 0.00000000e+00],
        [7.19892549e+01, 6.96532400e-01, 0.00000000e+00],
        [7.19892549e+01, 6.96532400e-01, 0.00000000e+00],
        [7.19892549e+01, 6.96532400e-01, 0.00000000e+00],
        [7.19892549e+01, 6.96532400e-01, 0.00000000e+00]],

       [[3.60741774e+02, 6.97315200e-01, 6.00000000e-02],
        [3.60741774e+02, 6.97315200e-01, 6.00000000e-02],
        [3.60741774e+02, 6.97315200e-01, 6.00000000e-02],
        [3.60741774e+02, 6.97315200e-01, 6.00000000e-02],
        [3.60741774e+02, 6.97315200e-01, 6.00000000e-02],
        [3.60741774e+02, 6.97315200e-01, 6.00000000e-02]],

       [[3.72992083e+02, 6.97310300e-01, 1.20000000e-01],
        [3.72992083e+02, 6.97310300e-01, 1.20000000e-01],
        [3.72992083e+02, 6.97310300e-01, 1.20000000e-01],
        [3.72992083e+02, 6.97310300e-01, 1.20000000e-01],
        [3.72992083e+02, 6.97310300e-01, 1.20000000e-01],
        [3

In [ ]:



## Because in some of the subjects the number of auditory tasks might differ from the visual ones
## we need to exploit 2 for loops, one for each here.
## This section is only for the auditory test
number_of_trials_per_run=30
audio_ones=np.ones(2*number_of_trials_per_run)
grip_ones=np.ones(number_of_trials_per_run)
for current_file in audio_files:
  task_file=current_file
  file_directory=audio_scans_folder+task_file
  temp=substring_after(file_directory,"run")
  run_index=temp[0]
  print("Now reading run number %s of the auditory test." %(run_index))
  current_run=sio.loadmat(file_directory)
  experiment_start_time=current_run['ExpStartTimeP']
  trial_start=current_run['stimulusStartTimeP']-experiment_start_time ##This is also grip start time
  grip_end_time=current_run['relaxStartTimeP']-experiment_start_time
  first_stimulus_start=current_run['SoundStartTimeP']-experiment_start_time
  first_stimulus_end=first_stimulus_start+0.1 ## Seems like we don't have the exact timings
  second_stimulus_start=first_stimulus_end+0.5
  second_stimulus_end=second_stimulus_start+0.1

  grip_length=grip_end_time-trial_start
  first_stimulus_length=first_stimulus_end-first_stimulus_start
  second_stimulus_length=second_stimulus_end-second_stimulus_start
  audio_first_vector=np.zeros(2*np.shape(first_stimulus_start)[1])
  audio_second_vector=np.zeros(2*np.shape(first_stimulus_start)[1])
  grip_first_vector=np.zeros(np.shape(trial_start)[1])
  grip_second_vector=np.zeros(np.shape(trial_start)[1])
  j=0
  for i in range(np.shape(first_stimulus_start)[1]):
    grip_first_vector[i]=trial_start[0,i]
    grip_second_vector[i]=grip_length[0,i]
    if j%2 ==0:
      audio_first_vector[j]=first_stimulus_start[0,i]
      audio_second_vector[j]=first_stimulus_length[0,i]
      j=j+1
    if j%2 !=0:
      audio_first_vector[j]=second_stimulus_start[0,i]
      audio_second_vector[j]=second_stimulus_length[0,i]
      j=j+1
  audio_ev=np.column_stack((audio_first_vector,audio_second_vector,audio_ones)) 
  grip_ev=np.column_stack((grip_first_vector,grip_second_vector,grip_ones)) 
  audio_filename=f"audio_run_{run_index}.txt"
  ev1_adress=save_dir_aud+audio_filename
  grip_filename=f"audio_grip_run_{run_index}.txt"
  ev2_adress=save_dir_aud+grip_filename
  fmt = ['%.4f', '%.4f', '%d']
  np.savetxt(ev1_adress,audio_ev, delimiter=" ",fmt=fmt)
  np.savetxt(ev2_adress,grip_ev, delimiter=" ",fmt=fmt)



Now reading run number 1 of the auditory test.
Now reading run number 2 of the auditory test.
Now reading run number 3 of the auditory test.
Now reading run number 4 of the auditory test.
Now reading run number 5 of the auditory test.


In [ ]:
first_stimulus_start

array([[  3.8099293,  16.0206326,  29.8313092,  42.2615242,  56.2621671,
         72.0135609,  87.7638165, 103.5049449, 119.2555947, 133.2561306,
        145.5063155, 159.5073296, 173.5076996, 185.758313 , 198.0091031,
        210.2594204, 224.2603517, 240.0115014, 252.2615588, 264.5121072,
        280.2530545, 294.2535608, 308.2540974, 322.2545207, 336.2554879,
        348.5062499, 362.5065597, 374.7572263, 387.0081127, 401.0084801]])

In [ ]:
visual_files

['subjectBAP103_Voddball_session2_run1_7_6_13_15.mat',
 'subjectBAP103_Voddball_session2_run2_7_6_13_25.mat',
 'subjectBAP103_Voddball_session2_run3_7_6_13_34.mat',
 'subjectBAP103_Voddball_session2_run4_7_6_13_42.mat',
 'subjectBAP103_Voddball_session2_run5_7_6_13_51.mat']

In [ ]:
import os
z=os.listdir(base_dir)
type(z)

list